In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


Initially the steps of preprocessing in OpenRefine will be analyzed, so they can be easily reproduced.

Firtly regarding the 'taylor_all_songs.csv' dataset.


*   Some observations of the data:
1.   Columns 'featuring', 'promotional release' have mostly NAs and 'single release' has many NAs
2.   key_mode = key_name + mode_name. One column derives from two differnt
3.   'Lyrics' column is empty
4.   Rows with many NAs in almost all numerical columns
5.   At the end, tracks with no album have some NAs related to album stuff
6.   There are some songs with different artist than taylor swift. In most of them she is featuring
*   Preprocess step1: Trim leading and endind whitespace + collapse consecutive whitespace(everywhere)
*   Step2: Convert these columns to number: track_number, danceability, energy, key, loudness, mode, speechiness, acousticness, instrumentalness, liveness, valence, tempo, time-signature, duration_ms
*   Step3: Remove 'lyrics' column
*   Step4: Convert these to date: album_release, promotional_release, single_release, track_release
*   Step5: 11 NAS in 'artist' + 'featuring'. We dont know if taylor swift is in this song. Because they exist in her albums and I searched them myself we convert the artist from NA to Taylor Swift exepct for two cases that there is NA in album_name and in many other fields as well. (exclude these two rows)
*   Step6: 11 rows with NAs in all the numerical columns, in single release in featuring. It also seems that it is the same 11 rows as before, in which 9 of them we imputed the artist to be Swift. There is a lot of information missing n these 11 rows that is impossible to correctly impute. Thus, I remove all of them. This means that the previous step5 was not needed, but it still mentioned here.




Now regarding the 'Taylor_Train.csv' dataset.
*   Step1: Trim leading and endind whitespace + collapse consecutive whitespace (everywhere)
*   Step2: One opening act in shangai is NA. I convert it to —, so we dont delete the row
*   Step3: 35 cases that both in attendance and revenue have — => Since those two are not needed in our assignment I am keeping the rows for the rest information they contain



In [21]:
# import libaries used through the project
import altair as alt
import pandas as pd
from vega_datasets import data
import geocoder
import time
import geopy
from geopy.geocoders import Nominatim

In [22]:
taylor_songs = pd.read_csv('/content/drive/MyDrive/Data_Vis/taylor-all-songs-cleaned_v1.csv') # read cleaned dataset

In [19]:
taylor_songs.head()

,album_name,ep,album_release,track_number,track_name,artist,featuring,bonus_track,promotional_release,single_release,...,liveness,valence,tempo,time_signature,duration_ms,explicit,key_name,mode_name,key_mode,duration_m
0,Taylor Swift,False,2006-10-24T00:00:00Z,1.0,Tim McGraw,Taylor Swift,NaN,False,NaN,2006-06-19T00:00:00Z,...,0.1210,0.425,76.009,4,232107,False,C,major,C major,3.86845
1,Taylor Swift,False,2006-10-24T00:00:00Z,2.0,Picture To Burn,Taylor Swift,NaN,False,NaN,2008-02-03T00:00:00Z,...,0.0962,0.821,105.586,4,173067,False,G,major,G major,2.88445
2,Taylor Swift,False,2006-10-24T00:00:00Z,3.0,Teardrops On My Guitar,Taylor Swift,NaN,False,NaN,2007-02-19T00:00:00Z,...,0.1190,0.289,99.953,4,203040,False,A#,major,A# major,3.38400
3,Taylor Swift,False,2006-10-24T00:00:00Z,4.0,A Place In This World,Taylor Swift,NaN,False,NaN,NaN,...,0.3200,0.428,115.028,4,199200,False,A,major,A major,3.32000
4,Taylor Swift,False,2006-10-24T00:00:00Z,5.0,Cold As You,Taylor Swift,NaN,False,NaN,NaN,...,0.1230,0.261,175.558,4,239013,False,F,major,F major,3.98355


In [23]:
taylor_train = pd.read_csv('/content/drive/MyDrive/Data_Vis/Taylor-Train-cleaned-v1.csv')  # read cleaned dataset
taylor_train.head()

,City,Country,Venue,Opening act(s),Attendance (tickets sold / available),Revenue,Tour
0,Evansville,United States,Roberts Municipal Stadium,Gloriana Kellie Pickler,"7,463 / 7,463","$360,617",Fearless_Tour
1,Jonesboro,United States,Convocation Center,Gloriana Kellie Pickler,"7,822 / 7,822","$340,328",Fearless_Tour
2,St. Louis,United States,Scottrade Center,Gloriana Kellie Pickler,"13,764 / 13,764","$650,420",Fearless_Tour
3,Alexandria,United States,Bishop Ireton High School,Gloriana Kellie Pickler,—,—,Fearless_Tour
4,North Charleston,United States,North Charleston Coliseum,Gloriana Kellie Pickler,"8,751 / 8,751","$398,154",Fearless_Tour


# Q1: Has the energy and acousticness of the albums changed along the years?

**Initially** I tried to solve this question by doing all the preprocessing before using altair. I did after immediatly with altair, but I keep my first trial in order you understand my process.
Also initally I did it only for the energy feature

In [25]:
print(len(taylor_songs['album_name'].dropna().unique()))  # drop rows that have nan values in at least one of this features
df_q1 = taylor_songs[['album_name','album_release','track_number', 'track_name', 'energy']].dropna()

13


In [26]:
df_q1 = df_q1.groupby(['album_name', 'album_release'])['energy'].mean().to_frame(name = 'energy').reset_index() # groupby album name release and energy
df_q1.sort_values('album_release',inplace=True)   # and sort by release date
df_q1.reset_index(inplace=True, drop=True)
df_q1

,album_name,album_release,energy
0,Taylor Swift,2006-10-24T00:00:00Z,0.664267
1,The Taylor Swift Holiday Collection,2007-10-14T00:00:00Z,0.496833
2,Fearless,2008-11-11T00:00:00Z,0.601053
3,Speak Now,2010-10-25T00:00:00Z,0.667000
4,Red,2012-10-22T00:00:00Z,0.600773
5,1989,2014-10-27T00:00:00Z,0.697063
6,reputation,2017-11-10T00:00:00Z,0.582867
7,Lover,2019-08-23T00:00:00Z,0.545222
8,folklore,2020-07-24T00:00:00Z,0.415824
9,evermore,2020-12-11T00:00:00Z,0.494059


In [27]:
# here is the chart after this preprocessing
alt.Chart(df_q1).mark_line().encode(
    x='album_release:T',
    y='energy:Q',
    tooltip = 'album_name'
).properties(title = 'Energy of albums through the years')

alt.Chart(...)

In the next cell I am plotting again in a line chart the energy through the years. The differences here is that I added points, titles, and colors. After I completed this chart I thought of having a dual axis chart with one more metric, being acousticness. I also rsolved the scale to combine these charts into one with a dual axis. I added grid=True as well, because the resolve_scale, deleted some parts of the grid. I put in axis labels the same colors with the corresponding line, so it is better interpeted from the user.

**Answer to Q1**: In chart_q1 you can se a line chart with dual axis, showing the evolution of energy(orange) and acousticness(blue) in the albums through the years. Energy and acousticness are in the y axis, while in x axis we have the release year of the album. There is also a tooltip to see the album each time.

In [28]:
ch1 = alt.Chart(taylor_songs).mark_line(point=True).encode(
    alt.X('album_release:T', axis=alt.Axis(title='Release year of album')),
    alt.Y('mean(energy):Q', scale=alt.Scale(domain=[0, 1]), axis=alt.Axis(title='Energy of album', titleColor='#FFAA00', grid=True)),
    tooltip = 'album_name',
    color=alt.value("#FFAA00")
).properties(title = 'Energy and acousticness of albums through the years')

ch2 = alt.Chart(taylor_songs).mark_line(point=True).encode(
    alt.X('album_release:T', axis=alt.Axis(title='Release year of album')),
    alt.Y('mean(acousticness):Q', scale=alt.Scale(domain=[0, 1]), axis=alt.Axis(title='Acousticness of album', grid=True, titleColor='#1f77b4')),
    tooltip = 'album_name'
).properties(title = 'Energy and acousticness of albums through the years')

chart_q1 = alt.layer(ch1, ch2).resolve_scale(y='independent')
chart_q1

alt.LayerChart(...)

# Q2: Is the energy of the albums constant among the songs?

In [29]:
dfq2 = taylor_songs[taylor_songs['album_name'].notnull()]  # keep only the songs thar exist in an album

Answering this directly with altair library. I wanted to create a chart that would contain information for all the songs of all albums. This one, was the first kind of chart I tried. I tried hard to find a better one, but I came to the conclusion that this is the best one I thought/searched and tried. In comments I keep my initial first try. In the final code, I have added opacity and size of marks, a scale range of x axis, along with titles and a specific pallete (dark2). This chart has in y axis all the albums and we can see the different danceability of every song in each album with the help of tooltip.

**Answer to Q1:** In chart_q2 you can see a scatterplot, that shows the danceability of each song in each album. This is created, by having the names of the albums in y axis and danceability in x axis. This way each album is not interfered with other albums and we can see the horizontal distribution of the danceability of the songs. The albums and their songs are colored, as can be seen from the legend in the right side.

In [30]:
# alt.Chart(dfq2).mark_point().encode(
#     alt.Y('danceability:Q')),
#     alt.X('album_name:N'),
#     tooltip = 'track_name',
#     color = 'album_name'
# ).properties(title = 'Danceability of the songs in each album')

chart_q2 = alt.Chart(dfq2).mark_point(opacity=0.7, shape='circle', size=20).encode(
    alt.X('danceability:Q', scale=alt.Scale(domain=[0.25, 0.95]), axis=alt.Axis(title='Danceability')),
    alt.Y('album_name:N', axis=alt.Axis(title='Album')),
    alt.Color('album_name', scale = alt.Scale(scheme = 'dark2'), title='Album'),
    tooltip = 'track_name'
).properties(title = 'Danceability of the songs in each album')

chart_q2

alt.Chart(...)

# Q3: Do new albums (Taylor’s version) change with respect to the original ones? (in respect of 'danceability', 'energy', 'speechiness', 'acousticness', 'liveness', 'valence' and duration in minutes)

In [31]:
# keep only the songs that exist in the albums Red, Fearless, in the original or in Taylor's version.
dfq3 = taylor_songs[taylor_songs['album_name'].isin(['Fearless', r"Fearless (Taylor's Version)", 'Red', r"Red (Taylor's Version)"])]
dfq3['album_name'].value_counts()

album_name
Red (Taylor's Version)         30
Fearless (Taylor's Version)    26
Red                            22
Fearless                       19
Name: count, dtype: int64

**I will present some things I tried** for this question. Initally I was trying to answer this question only for energy and I thought of creating bar charts, where for each song the metric is presented for every album. This was not a good idea because it required a lot of space, and it takes extra not needed space for songs that do not exist in the albums, like if a song belongs to album Red, the barchart reserves space for the metric of this song in album Fearless, but it doesnt exist there. This can be easily observed in a few cells.

In [32]:
dfq3['track_name2'] = dfq3['track_name'].apply(lambda x : x.split('(')[0].strip())  # keep the names of tracks without parenthesis. Not kept in the final chart

<ipython-input-32-51fde48cf8b5>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfq3['track_name2'] = dfq3['track_name'].apply(lambda x : x.split('(')[0].strip())  # keep the names of tracks without parenthesis. Not kept in the final chart


In [33]:
v = dfq3['track_name2'].value_counts()  # here I keep only songs that exist in at least two albums. Once again this is used only in my first not used trials.
dfq3b = dfq3[dfq3['track_name2'].isin(v.index[v.gt(1)])]

In [34]:
# Here it is clear that this was not a good idea
alt.Chart(dfq3).mark_bar().encode(
    x='album_name:N',
    y='energy:Q',
    color='album_name:N',
    column='track_name2:N'
).properties(
    width=40,
    height=200
)

alt.Chart(...)

In [35]:
# Second alternative was to create a stacked bar chart. It is surely better, but again it cant be that easily interpreted from user
alt.Chart(dfq3).mark_bar(
  ).encode(
  x=alt.X('track_name2:N'),
  y=alt.Y('energy:Q'),
  color='album_name:N'
)

alt.Chart(...)

Before creating a better chart for this, I did a **side quest**, to try to compare songs' energy in the 2 albums they belong with a connected dot plot. The chart is quite promising, but here I have only two songs for display. The way I do it, it would require more complicated code to do it for every song. However, I thought of keeping this and maybe expand it later in an interactive plot, where user could select which song he/she wants to check.

In [36]:
dfq3_sample = dfq3[dfq3['track_name2'].isin(['Breathe', 'Girl At Home'])]  # keep only two specific songs

In [37]:
ch1 = alt.Chart(dfq3_sample ).mark_circle(opacity = 1, size = 50, color = 'teal').encode(
    alt.X('energy:Q', scale=alt.Scale(domain=(0,1))),
    alt.Y('track_name2:N', sort='-x', axis=alt.Axis(title='Track Name')),
    alt.Color('album_name:N', title='Album')
).transform_filter(alt.datum.album_name == r"Fearless (Taylor's Version)").properties(title = r"Energy difference of original album and Taylor's version")

ch2 = alt.Chart(dfq3_sample ).mark_circle(opacity = 1, size = 50, color = 'lightsalmon').encode(
    alt.X('energy:Q', scale=alt.Scale(domain=(0,1))),
    alt.Y('track_name2:N', sort='-x', axis=alt.Axis(title='Track Name')),
    alt.Color('album_name:N', title='Album')
).transform_filter(alt.datum.album_name == "Fearless").properties(title = r"Energy difference of original album and Taylor's version")

ch4 = alt.Chart(dfq3_sample ).mark_circle(opacity = 1, size = 50, color = 'red').encode(
    alt.X('energy:Q', scale=alt.Scale(domain=(0,1))),
    alt.Y('track_name2:N', sort='-x', axis=alt.Axis(title='Track Name')),
    alt.Color('album_name:N', title='Album')
).transform_filter(alt.datum.album_name == r"Red (Taylor's Version)").properties(title = r"Energy difference of original album and Taylor's version")

ch5 = alt.Chart(dfq3_sample ).mark_circle(opacity = 1, size = 50, color = 'blue').encode(
    alt.X('energy:Q', scale=alt.Scale(domain=(0,1))),
    alt.Y('track_name2:N', sort='-x', axis=alt.Axis(title='Track Name')),
    alt.Color('album_name:N', title='Album')
).transform_filter(alt.datum.album_name == "Red").properties(title = r"Energy difference of original album and Taylor's version")

ch3 = alt.Chart(dfq3_sample).mark_line(color = 'black').encode(
    alt.X('energy:Q'),
    alt.Y('track_name2:N'),
    detail = 'track_name2',
).properties(title = r"Energy difference of original album and Taylor's version")

ch3 + ch1 + ch2 +ch4 + ch5

alt.LayerChart(...)

Going **back to answer the question**, I wanted to answer this for more than one metric, and kept 6 of them that I found interesting. So, I group by album name and taking the mean of these 6 metrics for each album. Below we can see the resulted kept dataframe.

In [38]:
dfq3c = dfq3.groupby('album_name', as_index=False)[['danceability', 'energy', 'speechiness', 'acousticness', 'liveness', 'valence']].mean()
dfq3c

,album_name,danceability,energy,speechiness,acousticness,liveness,valence
0,Fearless,0.575947,0.601053,0.031905,0.202448,0.159826,0.380053
1,Fearless (Taylor's Version),0.551038,0.639115,0.037854,0.214061,0.162354,0.421923
2,Red,0.633364,0.600773,0.036577,0.148799,0.126214,0.468191
3,Red (Taylor's Version),0.576967,0.586900,0.042250,0.176866,0.124780,0.424600


In [39]:
# In order to use this dataset in altair it would be easier if it was melted

dfq3c= dfq3c.melt(
   id_vars=['album_name'],
   value_vars=['danceability', 'energy', 'speechiness', 'acousticness', 'liveness', 'valence'],
   var_name='metric',
   value_name='value')
dfq3c.head()

,album_name,metric,value
0,Fearless,danceability,0.575947
1,Fearless (Taylor's Version),danceability,0.551038
2,Red,danceability,0.633364
3,Red (Taylor's Version),danceability,0.576967
4,Fearless,energy,0.601053


So, now I create a bar chart, that shows the mean value of each metric in these 4 albums kept. This could be the  final answer, but I decided to split it in two different charts, where the one has Red original and Taylor's and the other has Fearless original and Taylor's, so it is easier to compare how these albums changed when Taylor Swift decided to released them again. Note here that I use 'dark2' pallete again, like before, so the colors are similar. Also some modification of the title takes place with the help of TitleParams

In [40]:
alt.Chart(dfq3c).mark_bar().encode(
    alt.Color('album_name', scale = alt.Scale(scheme = 'dark2'), title='Album'),
    alt.X('album_name:N', title ='album'),
    alt.Y('value:Q', title='Value'),
    alt.Column('metric:N', title='')
).properties(title=alt.TitleParams (text= r"Difference of original and Taylor's version albums in various metrics", anchor='middle', subtitle="Metric", subtitleFontWeight="bold"))

alt.Chart(...)

In the next two cells, I am splitting the albums as I mentioned before. In order to do this, I do not alter the dataframe, but I use transform_filter and FildOneOfPredicate that allows me to keep only specific values of a specific column. This was a feature I learnt in the process.

In [41]:
chart_q3a = alt.Chart(dfq3c, width=alt.Step(25)).mark_bar().encode(
    alt.X('album_name:N', title='Album'),
    alt.Y('value:Q', title='value'),
    alt.Color('album_name:N', title='Album', scale = alt.Scale(scheme = 'dark2')),
    alt.Column('metric:N', title='Metric')
).transform_filter(
    alt.FieldOneOfPredicate(field='album_name', oneOf=[r"Red (Taylor's Version)","Red"])
).properties(title = r"Difference of album Red original and Red Taylor's version in various metrics")
chart_q3a

alt.Chart(...)

In [42]:
chart_q3b = alt.Chart(dfq3c, width=alt.Step(25)).mark_bar().encode(
    alt.X('album_name:N', title='Album'),
    alt.Y('value:Q', title='value'),
    alt.Color('album_name:N', title='Album', scale = alt.Scale(scheme = 'dark2')),
    alt.Column('metric:N', title='Metric')
).transform_filter(
    alt.FieldOneOfPredicate(field='album_name', oneOf=[r"Fearless (Taylor's Version)","Fearless"])
).properties(title = r"Difference of album Fearless original and Fearless Taylor's version in various metrics")
chart_q3b

alt.Chart(...)

Apart from the metrics I wanted to test also the difference of albums in duration. I did not do it in the same bar chart with the metrics, because the metrics are in the 0-1 scale and duration is not. So it would make the differences of the metrics seem not important. Also, I believe that converting duration in 0-1 scale as well, would not be easily interpreted from the user. Firstly, I create a new columns that contains the duration in minutes and not miliseconds. Then, for each album I sum the duration of all the songs.

In [43]:
taylor_songs['duration_m'] = taylor_songs['duration_ms']/60000
df3_min = taylor_songs[taylor_songs['album_name'].isin(["Fearless","Fearless (Taylor's Version)","Red (Taylor's Version)", "Red"])]
df3_min = df3_min.groupby('album_name', as_index=False) \
       .agg({'album_name': 'first', 'duration_m': 'sum'})
df3_min

,album_name,duration_m
0,Fearless,79.44380
1,Fearless (Taylor's Version),106.54175
2,Red,90.21775
3,Red (Taylor's Version),130.66400


The chart I chose was a simple dot plot. Initally I tried to do a connected dot plot, but I could not do it the way I had it in my mind. In this way, I think the comparison is easy.

In [44]:
ch1 = alt.Chart(df3_min).mark_circle(opacity = 1, size = 50, color = 'teal').encode(
    alt.X('duration_m:Q', title='Duration of album in minutes'),
    alt.Y('album_name:N', sort='-x', title='Album'),
).properties(title = r"Change in album duration of original and Taylor's version")

ch2 = alt.Chart(df3_min).mark_circle(opacity = 1, size = 50, color = 'lightsalmon').encode(
    alt.X('duration_m:Q'),
    alt.Y('album_name:N', sort='-x'),
).properties(title = r"Change in album duration of original and Taylor's version")

ch3 = alt.Chart(df3_min).mark_line(color = 'gray').encode(
    alt.X('duration_m:Q'),
    alt.Y('album_name:N'),
    detail = 'album_name'
).properties(title = r"Change in album duration of original and Taylor's version")

chart_q3c = ch3 + ch1 + ch2
chart_q3c

alt.LayerChart(...)

**Answer to Q3:** This question is answered with 3 charts. In the chart_q3a and chart_q3b you can see two bar charts, the first one compares Red original and Red Taylor's version albums in terms of acousticness, danceability, energy, liveness, speechiness and valence. While the latter, compares the same metrics but for albums Fearless and Fearless Taylor's version. Lastly, the chart_q3c completes this answer. In this chart we see a dot plot that compares the duration in total minutes between those 4 already mentioned albums. Below we see those 3 charts together.

In [46]:
c3 = alt.hconcat(chart_q3c, chart_q3a, chart_q3b).properties(title = {'text':'Taylor Swift released again the albums "Red" and "Fearless". How do they differ from the original ones?',
                                                                   'anchor':'middle', 'fontSize':18})
c3

alt.HConcatChart(...)

# Q4: Have the length of the albums changed along time?

In [47]:
dfq4 = taylor_songs[['album_name', 'album_release', 'track_name', 'duration_m']]
dfq4.head()

,album_name,album_release,track_name,duration_m
0,Taylor Swift,2006-10-24T00:00:00Z,Tim McGraw,3.86845
1,Taylor Swift,2006-10-24T00:00:00Z,Picture To Burn,2.88445
2,Taylor Swift,2006-10-24T00:00:00Z,Teardrops On My Guitar,3.38400
3,Taylor Swift,2006-10-24T00:00:00Z,A Place In This World,3.32000
4,Taylor Swift,2006-10-24T00:00:00Z,Cold As You,3.98355


To answer this question I create a simple line chart, by adding an extra calculated variable, being the sum. It should be noted here, that there are points with tooltips of the album name in the line chart, along with titles and also I increased the tickCount so, all the years from 2007 to 2022 can be seen.

**Answer to Q4:** In chart_q4 you can see a line chart that shows the evolution of the length of the album in minutes for each year and album from 2007 to 2022. In the y axis we have the total summed duration of each album in minutes and in x axis we have the years from 2007 to 2022. There are points and tooltips to indicate when the albums were released.

In [48]:
chart_q4 = alt.Chart(taylor_songs, width=alt.Step(35)).mark_line(point=True).encode(
    alt.X('album_release:T', title='Release year of album', axis=alt.Axis(tickCount=12)),
    alt.Y('sum(duration_m):Q', title='Duration of albums in minutes'),
    tooltip = 'album_name'
).properties(title = 'Length of albums in minutes through the years', width=450)
chart_q4

alt.Chart(...)

# Q5: Where does Tailor Swift has held a concert and how often?

In order to answer this question we need to use an **external data source** to take the longitude and latitude of the cities we have in our dataset. To be honest, collecting this information wasn't easy for me, because the 4 differnt sources I tried, were collecting lan and lon correctly, but they did not allow me to use it for multiple observations. They kept giving me Timeout error, due to believing I was a bot, or because they do not want to give so much information for free. In the next cells I am explaining how I achieved this.

In [ ]:
# Here I create a dataset with keeps the city with its country and the calculated frequency of Taylor's concerts in each city
concerts_by_city = taylor_train['City'].value_counts().reset_index()
concerts_by_city.columns = ['City', 'Frequency']
concerts_by_city = pd.merge(concerts_by_city, taylor_train[['City','Country']], how='left', on='City')
concerts_by_city = concerts_by_city.drop_duplicates().reset_index(drop=True)
concerts_by_city.head()

,City,Frequency,Country
0,Los Angeles,16,United States
1,London,12,England
2,Toronto,10,Canada
3,Philadelphia,10,United States
4,Melbourne,10,Australia


I am taking the coordinates of each city with the **geopy.geocoders, Nominatim **libray. These two functions in the next cell return the latitude and longitude of a given city, respectively. They are called later in the code. Because it was giving me TimeOut error as well, I put a time.sleep in the code, which means that it waits 10 seconds before asking coordinates of the next city in the data. This way it is harder to return a timeout error, but because I needed a backup, I was appending every new information coming with coordinates in a list, and if the code crashed I would continue from where I left. I bet there was some easier way to do it, but I did not manage to find it.

In [ ]:
def get_lat(x):
  geolocator = Nominatim(user_agent="Geopy Library")
  location = geolocator.geocode(x)
  lat = location.latitude
  return lat

def get_lon(x):
  geolocator = Nominatim(user_agent="http")
  location = geolocator.geocode(x)
  lng = location.longitude
  return lng


In [ ]:
columns = ['City', 'Country']
locations = (concerts_by_city[columns] + ' ').sum(axis=1).str.strip()  # create a list that has the city and the country, so the functions loop through this list.

In [ ]:
lats=[]
lons=[]

These next two cells in comments is the code of reading the coordinates for each city. They are in comments so, they are not accidentally pressed because  issues could occur. For this reason, after I completed this process, I saved the results in a csv file, that I will also provide to you. So, I read this csv file every time, instead of rerunning the code

In [ ]:
# for l in locations:
#   time.sleep(10)
#   print(l, get_lat(l))
#   lats.append(get_lat(l))

In [ ]:
# for l in locations:
#   time.sleep(10)
#   # print(l, get_lon(l))
#   lons.append(get_lon(l))

In [ ]:
# This print is the one the first complete run provided and the one saved in a csv.
concerts_by_city['latitude'] = lats
concerts_by_city['longitude'] = lons

,City,Frequency,Country,latitude,longitude
0,Los Angeles,16,United States,34.053691,-118.242766
1,London,12,England,51.507446,-0.127765
2,Toronto,10,Canada,43.653482,-79.383935
3,Philadelphia,10,United States,39.952724,-75.163526
4,Melbourne,10,Australia,-37.814245,144.963173


In [ ]:
concerts_by_city.to_csv('/content/drive/MyDrive/Data_Vis/concerts_by_city.csv', index=False)

Now I **read the csv** file every time.

In [49]:
concerts_by_city = pd.read_csv('/content/drive/MyDrive/Data_Vis/concerts_by_city.csv')

In [50]:
# For some reason Madrid, Spain did not return the correct coordinates and thus, I alter them myself
concerts_by_city.at[87, 'latitude'] = 40.416775
concerts_by_city.at[87, 'longitude'] = -3.703790
concerts_by_city[concerts_by_city['City']=='Madrid']

,City,Frequency,Country,latitude,longitude
87,Madrid,1,Spain,40.416775,-3.70379


Now we reached the chart creation part. I am reading the **world map** in which I display my data with the longitude and latitude information. There is also a tooltip with the city, and bubbles of the different frequencies for easier interpretation.

In [51]:
map = alt.topo_feature(data.world_110m.url, feature = 'countries')

world_map = alt.topo_feature("https://cdn.jsdelivr.net/npm/vega-datasets@v1.29.0/data/world-110m.json", 'countries')

# Create the base world map chart
worldChart = alt.Chart(world_map).mark_geoshape(
    fill='lightgray',
    stroke='white',
).properties(
    width=800,
    height=400
).project(
    "equirectangular"
)

map_plot = alt.Chart(concerts_by_city).mark_circle().encode(
    longitude='longitude:Q',
    latitude='latitude:Q',
    size='Frequency:Q',
    color=alt.Color(value='steelblue'),
    tooltip=['City:N','Frequency:Q']
).project(
    type="equirectangular"
).properties(title = 'Cities a concert was held by Taylor Swift and how many times')

chart_q5a = worldChart + map_plot
chart_q5a

alt.LayerChart(...)

Because most of the concerts happened in USA, I display again but this time in a map **only** with the **USA**. I use transform filter and FieldEqualPredicate so only those cities that have 'United States' as a country are kept.

In [52]:
map = alt.topo_feature(data.us_10m.url, feature = 'states')

usChart = alt.Chart(map).mark_geoshape(
    fill = 'lightgray',
    stroke = 'white',
).properties(
    width = 400,
    height = 300
).project('albersUsa')

map_plot = alt.Chart(concerts_by_city).mark_circle().encode(
    longitude='longitude:Q',
    latitude='latitude:Q',
    size='Frequency:Q',
    color=alt.Color(value='steelblue'),
    tooltip=['City:N','Frequency:Q']
).transform_filter(
    alt.FieldEqualPredicate(field='Country', equal="United States")
).properties(title = 'Cities in USA a concert was held by Taylor Swift and how many times')

chart_q5b = usChart + map_plot
chart_q5b

alt.LayerChart(...)

**Answer to Q5:** This question is answered with two charts. The first one, chart_q5a shows the world map with all the places Taylor Swift had concerts along with the frequency shown be legend and tooltip. The second chart, chart_q5b does the same job, but only for the United States of America, due to more concerts happenning there, in various cities.

In [55]:
c5 = alt.hconcat(chart_q5a, chart_q5b).properties(title = {'text':'Which are the cities around the world that Taylor Swift had concerts and with what frequency?',
                                                                   'anchor':'middle', 'fontSize':18})
c5

alt.HConcatChart(...)

# Final-multi view visualization.

Here is the final multi-view visualization. It contains 8 chosen charts from the previous 5 questions, along with some text for better interpretation. I will mention two issues I had that I could not solve. Firstly, I could not find a way to change the locations of the Frequency legends for the map charts. Secondly and most important, altair changes by default the colors on the two bar charts. I gave them teh default (bue and orange) colors and then the 'dark2' pallete like in the danceability of the songs graph, but when displaying them before they had different colors and the same colors among them. Now altair gave them different in the concatenation. I tried a lot to fing a way to change the colors in the multi-view chart, but I did not manage it. Maybe, I just have to trust altair!

In [56]:
c1 = alt.hconcat(chart_q1, chart_q4, chart_q2).properties(title = {'text':'How did her albums change along the years regarding energy, acousticness and length? Is the danceability of each album constant or varying?',
                                                          'subtitle' : ['Energy constant with some steep changes lately, while acousticness had a huge increase around 2020 and dropped after.',
                                                                        'Energy and acousticness seem to be antidiametric.',
                                                                        'Length had some alternations with a steep increase and decrease righ after lately as well.',
                                                                        'The danceability not constatnt. In almost all many songs in the middle, but all of them have a variety (high and low). "Reputation" an exepction.'],
                                                                   'anchor':'middle', 'fontSize':18})
c2 = alt.hconcat(chart_q3a, chart_q3b, chart_q3c).properties(title = {'text':'Taylor Swift released again the albums "Red" and "Fearless". How do they differ from the original ones?',
                                                          'subtitle' : ['Her "Red" version changed slightly from the original in most metrics, but her version had a small decrease in danceability and valence.',
                                                                        '"Fearless" had noticeable increase in energy and valence from the original and a small decrease in danceability.',
                                                                        ' Taylor\'s "Red" lasts 40 mins more, while "Fearless" lasts around 26 mins more from the originals.'],
                                                                   'anchor':'middle', 'fontSize':18})
c3 = alt.hconcat(chart_q5a, chart_q5b).properties(title = {'text':'Which are the cities around the world that Taylor Swift had concerts and with what frequency?',
                                                          'subtitle' : ['It is obvious that most of them took place in USA in various cities, with 16 only in Los Angeles.',
                                                                        'There are concerts in Australia and some specific cities in Eastern Asia.',
                                                                        'While there are a bunch of concerts in Europe the vast majority are in the United Kingdom.',
                                                                        'South America, Africa and the biggest part of Asia had 0 concerts.'],
                                                                   'anchor':'middle', 'fontSize':18})
c4 = alt.vconcat(c1, c3)
chart = alt.vconcat(c4, c2).properties(title = {'text':'Evolution of Taylor\'s Swift songs, albums and concerts along the years', 'anchor':'middle', 'fontSize':28})
chart

alt.VConcatChart(...)